In [1]:
import numpy as np

In [2]:
def initialization(next_layer_dims, prev_layer_dims):
    W  = np.random.randn(next_layer_dims, prev_layer_dims)
    b = np.zeros([next_layer_dims,1])
    return W, b

In [25]:
def forward_prop(A_prev, W, b):
    Z = np.dot(W, A_prev)+b
    
    return Z

In [4]:
def activation_function(Z, activation):
    if activation == 'sigmoid':
        A = 1/(1+np.exp(-Z))

    if activation == 'relu':
        A = np.maximum(Z, 0)

    return A 

In [5]:
def cost_function(AL, label):

    m = AL.shape[1]
    cost = -1/m*(np.sum(label*np.log(AL)+(1-label)*np.log(1-AL)))
    assert(cost.shape == ())

    return cost

In [6]:
def relu_grad(dA, A):
    dZ = np.array(dA, copy=True)
    print(A.shape)
    print(dZ.shape[1])
  
    for i in range(dZ.shape[1]):
        if A[i] == 0:
            dZ[i] = 0
     
    return dZ
     

In [7]:
def sigmoid_grad(A):
    return A*(1-A)

In [8]:
def backward_prop(dZ, A_prev, W ):

    m = A_prev.shape[1]
    dW = 1/m*(np.dot(dZ, A_prev.T))
    db = 1/m*np.sum(dZ)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

    #dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

In [26]:
def training_model(no_of_layers, nL, X, Y, learning_rate, EPOCHS):

    param= {}
    results={"A0":X}
    
    Z = {}
    #initializing parameters:
    for l in range(1, no_of_layers):
        param["W" + str(l)], param["b" + str(l)] = initialization(nL[l],nL[l-1])        

    for epochs in range(EPOCHS):
        for l in range(1, no_of_layers-1):
            
            #initialization of parameters 
            W = param["W" + str(l)]
            b = param["b" + str(l)]

            #forward propogation 
            Z["Z"+ str(l)] = forward_prop(results["A" + str(l-1)], W, b)
            results["A"+ str(l)] = activation_function( Z["Z"+str(l)], activation="relu")

        Z["Z" + str(no_of_layers-1)] = forward_prop(results["A" + str(no_of_layers-2 )], param["W"+str(no_of_layers-1)], param["b"+str(no_of_layers-1)])
        results["A"+ str(no_of_layers-1)] = activation_function( Z["Z"+ str(no_of_layers-1)], activation="sigmoid")

        #finding cost:

        cost = cost_function(results["A"+ str(no_of_layers-1)], Y)

        print(cost)
        #backward prop :
        grads = {}

        AL = results["A"+ str(no_of_layers-1)]
        print(AL.shape)
        grads["dA"+str(no_of_layers-1)] = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        dZ = grads["dA"+str(no_of_layers-1)]*sigmoid_grad(AL)
        for l in range(no_of_layers-1,0,-1):
            grads["dA"+str(l)],  grads["dW"+str(l)], grads["db"+str(l)] = backward_prop(dZ, AL, param["W"+str(l)] )
            dZ = relu_grad(grads["dA"+str(l)],results["A"+str(l)])
        # updating the parameters:

        for l in range(1,no_of_layers):
            param["W"+str(l)] = param["W"+str(l)] - learning_rate*grads["dW"+str(l)]
            param["b"+str(l)] = param["b"+str(l)] - learning_rate*grads["db"+str(l)] 
            
    return param


In [27]:
def predict(X, Y, param,no_of_layers):
    for l in range(1, no_of_layers):
            
        #initialization of parameters 
        W = param["W" + str(l)]
        b = param["b" + str(l)]

        #forward propogation 
        results["A"+ str(l)] = forward_prop(results["A" + str(l-1)], W, b)
        results["A"+ str(l)] = activation_function( results["A"+ str(l)], activation="relu")

    results["A" + str(no_of_layers)] = forward_prop(results["A" + str(no_of_layers-1 )], param["W"+str(no_of_layers)], param["b"+str(no_of_layers)])
    results["A"+ str(no_of_layers)] = activation_function( results["A"+ str(no_of_layers)], activation="sigmoid")

    correct_count = 0
    total_count = len(Y)

    for i in results["A"+ str(no_of_layers)]:
        if i>0.5:
            i=1
        else:
            i=0
    
    for i in range(len(Y)):
        if Y[i] == results["A"+ str(no_of_layers)][i]:
            correct_count+=1

    accuracy = correct_count/total_count

    print(f"accuracy: {accuracy}")
    
    cost = cost_function(results["A"+ str(no_of_layers)], Y)
    print(f"cost:{cost}")



In [28]:
import h5py
import scipy
%load_ext autoreload
%autoreload 2
from dnn_app_utils_v3 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

train_x_orig.shape

(209, 64, 64, 3)

In [30]:
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

train_x.shape[1]

209

In [31]:
nL = [train_x.shape[1], 4,i 3, 1]

params = training_model(4, nL, train_x, train_y, 0.01, 5)

ValueError: shapes (4,209) and (12288,209) not aligned: 209 (dim 1) != 12288 (dim 0)